In [101]:
import torch
import dgl
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric
from torch_geometric.data import HeteroData
import dgl.data


In [21]:
import dgl.data
dataset = dgl.data.FraudYelpDataset()
dataset

Extracting file to C:\Users\bin\.dgl\yelp
Done saving data into cached files.


TypeError: object of type 'DGLHeteroGraph' has no len()

In [22]:
dataset[0].number_of_nodes()

45954

In [14]:
dataset[0]

Graph(num_nodes=716847, num_edges=13954819,
      ndata_schemes={'feat': Scheme(shape=(300,), dtype=torch.float32), 'label': Scheme(shape=(100,), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool), 'test_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={})

In [24]:
dataset[0].ntypes

['review']

In [25]:
dataset[0].etypes

['net_rsr', 'net_rtr', 'net_rur']

In [30]:
graph = dataset[0]

In [32]:
graph.ndata['feature'].shape

torch.Size([45954, 32])

In [34]:
graph.ndata['label'].shape

torch.Size([45954])

In [35]:
graph.__class__

dgl.heterograph.DGLHeteroGraph

In [77]:
graph.etypes

['net_rsr', 'net_rtr', 'net_rur']

In [88]:
for etype in graph.etypes:
    print(f"{etype} : {graph.num_edges(etype)}")
# 如果图只有一种节点或边类型，则不需要明确类型


net_rsr : 6805486
net_rtr : 1147232
net_rur : 98630


In [90]:
graph.num_nodes()

45954

![solution](./pic/solution.png)

In [48]:
# graph_pyg = (torch_geometric.utils.from_networkx(dgl.to_networkx(dgl.to_homogeneous(graph)))).to_heterogeneous()
# torch.save(graph_pyg,'graph_pyg')

### construct a PyG dataset from a DGL one 

In [57]:
graph

Graph(num_nodes={'review': 45954},
      num_edges={('review', 'net_rsr', 'review'): 6805486, ('review', 'net_rtr', 'review'): 1147232, ('review', 'net_rur', 'review'): 98630},
      metagraph=[('review', 'review', 'net_rsr'), ('review', 'review', 'net_rtr'), ('review', 'review', 'net_rur')])

In [64]:
graph.etypes

['net_rsr', 'net_rtr', 'net_rur']

In [99]:
graph.adj(etype='net_rsr').shape

torch.Size([45954, 45954])

In [107]:
data = HeteroData()

data['reviews'].x = graph.ndata['feature']
data['reviews'].y = graph.ndata['label']
data['reviews'].train_mask = graph.ndata['train_mask']
data['reviews'].val_mask = graph.ndata['val_mask']
data['reviews'].test_mask = graph.ndata['test_mask']
data['review', 'net_rsr', 'review'].edge_index = graph.adj(etype='net_rsr')
data['review', 'net_rtr', 'review'].edge_index = graph.adj(etype='net_rtr')
data['review', 'net_rur', 'review'].edge_index = graph.adj(etype='net_rur')



In [108]:
data

HeteroData(
  reviews={
    x=[45954, 32],
    y=[45954],
    train_mask=[45954],
    val_mask=[45954],
    test_mask=[45954]
  },
  (review, net_rsr, review)={ edge_index=[45954, 45954] },
  (review, net_rtr, review)={ edge_index=[45954, 45954] },
  (review, net_rur, review)={ edge_index=[45954, 45954] }
)

In [109]:
torch.save(data,'graph_pyg')

In [111]:
data.metadata()

(['reviews'],
 [('review', 'net_rsr', 'review'),
  ('review', 'net_rtr', 'review'),
  ('review', 'net_rur', 'review')])